In [ ]:
# Melakukan impor libraries yang diperlukan untuk visualisasi data.
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.gridspec as gridspec
import math
import cv2
import time
import datetime

In [ ]:
# Melakukan impor MTCNN untuk deteksi wajah.
! pip install mtcnn
from mtcnn.mtcnn import MTCNN

In [ ]:
# Mencatat waktu dimulainya keseluruhan program visualisasi data.
global_start_time = time.time()

In [ ]:
# Membaca sekaligus menampilkan isi dari data training.
df_train = pd.read_csv('../input/bdc-satriadata2021/train.csv')
df_train

In [ ]:
# Membaca sekaligus menampilkan id dari data testing.
df_test = pd.read_csv('../input/bdc-satriadata2021/submission.csv')
df_test

In [ ]:
# Mencatat waktu dimulainya pembacaan training images.
start_time = time.time()

# Mencatat index dari setiap training images.
img_train = {}
index_train = list(df_train.iloc[:,0])
subindex_train = [1, 2, 3]
filename_master_train = '../input/bdc-satriadata2021/Training/'

for index in index_train:
    # Membaca dan menyimpan training images.
    img_train[index] = {}
    img_index = str(index) + '/' + str(index) + '_'
    for subindex in subindex_train:
        img_subindex = str(subindex) + '.jpg'
        filename_train = filename_master_train + img_index + img_subindex
        img_train[index][subindex] = mpimg.imread(filename_train)

# Mencatat waktu berakhirnya pembacaan training images.
end_time = time.time()

# Menampilkan waktu eksekusi dari pembacaan training images.
execution_time = datetime.timedelta(seconds = end_time - start_time)
print("execution time: %s" % (execution_time))

In [ ]:
# Mencatat waktu dimulainya pembacaan testing images.
start_time = time.time()

# Mencatat index dari setiap testing images.
img_test = {}
index_test = list(df_test.iloc[:,0])
filename_master_test = '../input/bdc-satriadata2021/Testing/'

for index in index_test:
    # Membaca dan menyimpan testing images.
    img_test[index] = {}
    img_index = index + '.jpg'
    filename_test = filename_master_test + img_index
    img_test[index] = mpimg.imread(filename_test)

# Mencatat waktu berakhirnya pembacaan testing images.
end_time = time.time()

# Menampilkan waktu eksekusi dari pembacaan training images.
execution_time = datetime.timedelta(seconds = end_time - start_time)
print("execution time: %s" % (execution_time))

In [ ]:
# Mendeklarasikan pendeteksi wajah.
detector = MTCNN()

In [ ]:
# Menentukan batas kepercayaan dan ukuran deteksi wajah.
conf_threshold = 0.99
conf_precision = 6
size_threshold = 0.7

In [ ]:
# Mencatat waktu dimulainya visualisai training images.
start_time = time.time()

# Menyiapkan ukuran visualisasi data.
nrows = df_train.shape[0]
ncols = 3

# Menyiapkan tempat untuk penulisan judul.
title_frame = plt.figure(figsize = (6 * ncols, 10e-03), constrained_layout = False)
title_frame.patch.set_facecolor('white')
title_spec = gridspec.GridSpec(1, 1, figure = title_frame)

# Menuliskan judul.
main_title = title_frame.add_subplot(title_spec[0, 0])
main_title.set(xticks = [], yticks = [])
main_title.set_title('TRAINING IMAGES', fontsize = 72, color = '#007BA7')

for row in range(nrows):
    # Menyiapkan tempat untuk visualisasi data.
    frame = plt.figure(figsize = (6 * ncols, 8), constrained_layout = False)
    frame.patch.set_facecolor('white')
    spec = gridspec.GridSpec(1, ncols, figure = frame)

    for col in range(ncols):
        # Menyiapkan index dan data yang diperlukan untuk setiap training image.
        nomor = df_train.iloc[row][0]
        urutan_gambar = col + 1
        jenis_kelamin = ['perempuan', 'laki-laki'][df_train.iloc[row][1]]
        usia = df_train.iloc[row][2]
        
        # Mendeteksi wajah.
        my_image = img_train[nomor][urutan_gambar].copy()
        face_location = detector.detect_faces(my_image)
        face_counter = 0
        max_conf = 0
        min_conf = 0
        
        if len(face_location):
            # Menyeleksi wajah yang akan ditampilkan.
            size_box = []
            conf_box = []
            for element in range(len(face_location)):
                size_box.append(max(face_location[element]['box'][2], face_location[element]['box'][3]))
                conf_box.append(face_location[element]['confidence'])
            face_table = pd.DataFrame([size_box, conf_box]).T
            face_table[0] = face_table[0] / max(face_table[0])
            face_table = face_table[(face_table[0] > size_threshold) & (face_table[1] > conf_threshold)]
            face_priority_index = list(face_table.sort_values(0, ascending = False).index)
            if not len(face_priority_index):
                face_priority_index = [0]
            
            # Memberi batas area wajah pada wajah yang terpilih.
            for element in range(len(face_priority_index)): 
                # Warna biru untuk wajah selain wajah terbesar.
                frame_color = (0, 0, 255)
                
                # Warna merah untuk wajah terbesar.
                if not element:
                    frame_color = (255, 0, 0)
                    max_conf = face_location[face_priority_index[element]]['confidence']
                    min_conf = face_location[face_priority_index[element]]['confidence']
                
                max_conf = max(max_conf, face_location[face_priority_index[element]]['confidence'])
                min_conf = min(min_conf, face_location[face_priority_index[element]]['confidence'])
                face_counter += 1
                x_coordinate, y_coordinate, width, height = face_location[face_priority_index[element]]['box']
                cv2.rectangle(my_image, (x_coordinate, y_coordinate),
                              (x_coordinate + width, y_coordinate + height), frame_color, 10)
        
        # Memunculkan gambar.
        add_image = frame.add_subplot(spec[0, col])
        add_image.imshow(my_image)
        
        # Memunculkan judul.
        title = 'Gambar ' + str(nomor) + '_' + str(urutan_gambar)
        add_image.set_title(title, fontsize = 14)
        
        # Memunculkan label.
        label_0 = 'jenis kelamin: ' + jenis_kelamin + '\n'
        label_1 = 'usia: ' + str(usia) + ' tahun' + '\n'
        label_2 = 'wajah terdeteksi: ' + str(face_counter) + '\n'
        label_3 = 'kepercayaan maksimum: ' + str(100 * math.floor(max_conf*10**conf_precision)/10**conf_precision) + '%' + '\n'
        label_4 = 'kepercayaan minimum: ' + str(100 * math.floor(min_conf*10**conf_precision)/10**conf_precision) + '%'
        add_image.set_xlabel(label_0 + label_1 + label_2 + label_3 + label_4, fontsize = 12)
        add_image.set(xticks = [], yticks = [])
    
    # Menampilkan keseluruhan visualisasi data.
    plt.show()

# Mencatat waktu berakhirnya visualisasi training images.
end_time = time.time()

# Menampilkan waktu eksekusi dari visualisasi training images.
execution_time = datetime.timedelta(seconds = end_time - start_time)
print("execution time: %s" % (execution_time))

In [ ]:
# Mencatat waktu dimulainya visualisai testing images.
start_time = time.time()

# Menyiapkan ukuran visualisasi data.
nrows = df_test.shape[0] // 3
ncols = 3

# Menyiapkan tempat untuk penulisan judul.
title_frame = plt.figure(figsize = (6 * ncols, 10e-03), constrained_layout = False)
title_frame.patch.set_facecolor('white')
title_spec = gridspec.GridSpec(1, 1, figure = title_frame)

# Menuliskan judul.
main_title = title_frame.add_subplot(title_spec[0, 0])
main_title.set(xticks = [], yticks = [])
main_title.set_title('TESTING IMAGES', fontsize = 72, color = '#007BA7')

for row in range(nrows):
    # Menyiapkan tempat untuk visualisasi data.
    frame = plt.figure(figsize = (6 * ncols, 8), constrained_layout = False)
    frame.patch.set_facecolor('white')
    spec = gridspec.GridSpec(1, ncols, figure = frame)

    for col in range(ncols):
        # Menyiapkan index untuk setiap testing image.
        index = df_test.iloc[3 * row + col, 0]
        
        # Mendeteksi wajah.
        my_image = img_test[index].copy()
        face_location = detector.detect_faces(my_image)
        face_counter = 0
        max_conf = 0
        min_conf = 0
        
        if len(face_location):
            # Menyeleksi wajah yang akan ditampilkan.
            size_box = []
            conf_box = []
            for element in range(len(face_location)):
                size_box.append(max(face_location[element]['box'][2], face_location[element]['box'][3]))
                conf_box.append(face_location[element]['confidence'])
            face_table = pd.DataFrame([size_box, conf_box]).T
            face_table[0] = face_table[0] / max(face_table[0])
            face_table = face_table[(face_table[0] > size_threshold) & (face_table[1] > conf_threshold)]
            face_priority_index = list(face_table.sort_values(0, ascending = False).index)
            if not len(face_priority_index):
                face_priority_index = [0]
            
            # Memberi batas area wajah pada wajah yang terpilih.
            for element in range(len(face_priority_index)): 
                # Warna biru untuk wajah selain wajah terbesar.
                frame_color = (0, 0, 255)
                
                # Warna merah untuk wajah terbesar.
                if not element:
                    frame_color = (255, 0, 0)
                    max_conf = face_location[face_priority_index[element]]['confidence']
                    min_conf = face_location[face_priority_index[element]]['confidence']
                
                max_conf = max(max_conf, face_location[face_priority_index[element]]['confidence'])
                min_conf = min(min_conf, face_location[face_priority_index[element]]['confidence'])
                face_counter += 1
                x_coordinate, y_coordinate, width, height = face_location[face_priority_index[element]]['box']
                cv2.rectangle(my_image, (x_coordinate, y_coordinate),
                              (x_coordinate + width, y_coordinate + height), frame_color, 10)
        
        # Memunculkan gambar.
        add_image = frame.add_subplot(spec[0, col])
        add_image.imshow(my_image)
        
        # Memunculkan judul.
        title = str(3 * row + col + 1) + '. ' + index
        add_image.set_title(title, fontsize = 12)
        add_image.set(xticks = [], yticks = [])
        
        # Memunculkan label.
        label_0 = 'wajah terdeteksi: ' + str(face_counter) + '\n'
        label_1 = 'kepercayaan maksimum: ' + str(100 * math.floor(max_conf*10**conf_precision)/10**conf_precision) + '%' + '\n'
        label_2 = 'kepercayaan minimum: ' + str(100 * math.floor(min_conf*10**conf_precision)/10**conf_precision) + '%'
        add_image.set_xlabel(label_0 + label_1 + label_2, fontsize = 12)
        add_image.set(xticks = [], yticks = [])
    
    # Menampilkan keseluruhan visualisasi data.
    plt.show()

# Mencatat waktu berakhirnya visualisasi testing images.
end_time = time.time()

# Menampilkan waktu eksekusi dari visualisasi testing images.
execution_time = datetime.timedelta(seconds = end_time - start_time)
print("execution time: %s" % (execution_time))

In [ ]:
# Mencatat waktu berakhirnya keseluruhan program visualisasi data.
global_end_time = time.time()

# Menampilkan waktu eksekusi dari keseluruhan program visualisasi data.
total_execution_time = datetime.timedelta(seconds = global_end_time - global_start_time)
print("total execution time: %s" % (total_execution_time))